In [1]:
from trialstreamer import minimap

In [2]:
from psycopg2.extras import execute_values
from trialstreamer import dbutil, config
import psycopg2
import trialstreamer
from trialstreamer.pubmed import grouper
import json
import tqdm


[INFO] robotdata 2019-05-07 11:25:05,716: Welcome to RobotData :)


In [3]:
read_cur = dbutil.db.cursor(cursor_factory=psycopg2.extras.RealDictCursor)
update_cur = dbutil.db.cursor()

In [4]:
def get_unique_terms(l):
    
    terms = [minimap.minimap(s) for s in l]
    flat_terms = [item for sublist in terms for item in sublist]
    
    encountered_terms = set()
    unique_terms = []
    
    for term in flat_terms:
        if term['cui'] not in encountered_terms:
            term.pop('start_idx')
            term.pop('end_idx')
            term.pop('source_text')
            unique_terms.append(term)
            encountered_terms.add(term['cui'])
        
    
    return unique_terms
    
    
    

In [5]:

read_cur.execute("SELECT * FROM pubmed_pico;")

records = read_cur.fetchall()

batch_size = 100


for batch in tqdm.tqdm(grouper(records, batch_size), desc='articles annotated'):
    
    out = []
        
    
    for r in batch:
        pmid = r['pmid']
        population_mesh = get_unique_terms(r['population'])
        interventions_mesh = get_unique_terms(r['interventions'])
        outcomes_mesh = get_unique_terms(r['outcomes'])
        
        row = (json.dumps(population_mesh),
               json.dumps(interventions_mesh),
               json.dumps(outcomes_mesh),
               pmid)
        
        out.append(row)
        
        update_cur.execute("update pubmed_pico set population_mesh=(%s), interventions_mesh=(%s), outcomes_mesh=(%s) where pmid=(%s);",
    row)
        
    dbutil.db.commit()


        
    
    
    


articles annotated: 6500it [9:09:43,  4.30s/it]


In [7]:
QUERY FOR OBTAINING TRIALS BY POPULATION
select pmid from pubmed_pico, jsonb_array_elements(population_mesh) as p where p->>'mesh_term'='Stroke' limit 10;


In [6]:
ids = ['NCT01657461',
 'NCT01270867',
'NCT01054560',
 'ACTRN12611000969965',
 'NCT01492725',
 'NCT01778335',
 'NCT01657461',
 'NCT01054560',
 'NCT01327989',
 'NCT02523261',
 'NCT01062698',
 'NCT03036631',
 'NCT01692379',
 'NCT02135926',
 'NCT01692379',
 'NCT01745692',
 'NCT02126085',
 'NCT02157532',
 'NCT01692379',
 'NCT00860366',
 'NCT02586415',
 'NCT02350283',
 'NCT01062698',
 'NCT01657461',
 'NCT01584609',
 'NCT02222714']




In [7]:
set(ids)

{'ACTRN12611000969965',
 'NCT00860366',
 'NCT01054560',
 'NCT01062698',
 'NCT01270867',
 'NCT01327989',
 'NCT01492725',
 'NCT01584609',
 'NCT01657461',
 'NCT01692379',
 'NCT01745692',
 'NCT01778335',
 'NCT02126085',
 'NCT02135926',
 'NCT02157532',
 'NCT02222714',
 'NCT02350283',
 'NCT02523261',
 'NCT02586415',
 'NCT03036631'}